In [1]:
!pip install meteostat

In [2]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from meteostat import Point, Daily
import holidays
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
import datetime

# Set plot style
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load Data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [4]:
train_df['id'] = pd.to_datetime(train_df['id'])  # Ensure 'id' column is datetime


In [5]:
train_df

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6
...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4


### 2. Preprocessing

#### Date and Time Features
- [ ] **Extract Date Features**: Extract year, month, day, and hour from the timestamp for each entry.
- [ ] **Cyclic Features**: Add cyclic features for hour and month using sine and cosine transformations.

#### Lag and Rolling Features
- [ ] **Lagged Values**: Create lagged features for each pollutant (1, 24, 168 hours to capture short, daily, and weekly patterns).
- [ ] **Rolling Statistics**: Add rolling mean, rolling standard deviation, and rolling min/max features to smooth trends.

#### Handle Missing Values
- [ ] **Impute Missing Values**: Choose and implement a strategy (e.g., forward-fill, backward-fill, interpolation) for missing data in each pollutant.
- [ ] **Drop or Flag Missing Rows**: If a significant portion of the data is missing, consider dropping those rows or creating an indicator feature to mark them.

#### External Weather Data (Optional but Recommended)
- [ ] **Incorporate Weather Data**: If available, obtain weather data for Paris (temperature, humidity, wind speed).
- [ ] **Merge with Pollutant Data**: Align weather data with your time series dataset based on the timestamp.

#### Data Scaling
- [ ] **Scale Features**: Use standard scaling or min-max scaling for numerical features, especially for pollutants and weather variables (if any).

#### Target Preparation
- [ ] **Prepare Target Variables**: Shift the pollutant columns to create target variables aligned with the forecasting horizon (e.g., next hour, next day).
- [ ] **Split Data**: Divide the data into training and validation sets, ensuring the split respects the time sequence.




# Extracting Date Information
for df in [train_df, test_df]:
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['day_of_week'] = df.index.dayofweek
    df['hour'] = df.index.hour

# Create Lag Features (e.g., lags of 1, 3, 6, and 24 hours)
lags = [1, 3, 6, 24]
for lag in lags:
    for col in ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']:
        train_df[f'{col}_lag_{lag}'] = train_df[col].shift(lag)

# Drop rows with NaN values introduced by lags (at the beginning of the data)
train_df.dropna(inplace=True)

# Scaling/Normalization

# Select features for scaling (all pollutant columns and lags)
scaler = StandardScaler()
pollutant_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'] 
pollutant_columns = pollutant_columns + [f'{col}_lag_{lag}' for col in pollutant_columns for lag in lags]
train_df[pollutant_columns] = scaler.fit_transform(train_df[pollutant_columns])

# Weather

In [6]:
# Adding External Weather Data

# Define the time period for which we need the data
start_date = train_df.index.min()
end_date = train_df.index.max()

# Define the location (Paris coordinates)
paris = Point(48.8566, 2.3522)

# Fetch daily historical data for Paris from Meteostat
weather_data = Daily(paris, start_date, end_date)
weather_data = weather_data.fetch()

# Display the first few rows of the weather data
print(weather_data.head())

            tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt  pres  tsun
time                                                                  
1900-01-01   NaN   4.5  10.1   0.0   NaN   NaN   NaN   NaN   NaN   NaN
1900-01-02   NaN   7.9  12.0   7.5   NaN   NaN   NaN   NaN   NaN   NaN
1900-01-03   NaN   6.0   9.3   2.2   NaN   NaN   NaN   NaN   NaN   NaN
1900-01-04   NaN   6.0   8.3   3.1   NaN   NaN   NaN   NaN   NaN   NaN
1900-01-05   NaN   2.2   6.0   0.0   NaN   NaN   NaN   NaN   NaN   NaN


In [7]:
# Reset index in `weather_data` to use the date as a regular column
weather_data = weather_data.reset_index()
weather_data['weather_time'] = weather_data['time'].dt.floor('D')  # Convert to daily frequency

train_df = train_df.reset_index()
train_df['air_quality_time'] = train_df['id'].dt.floor('D')

# Merge the weather data with your air quality data
merged_df = pd.merge(train_df, weather_data, left_on='air_quality_time', right_on='weather_time', how='left')

# Drop unnecessary columns and clean up
merged_df = merged_df.drop(columns=['air_quality_time', 'weather_time'])

# Save the combined DataFrame to a new CSV file
merged_df.to_csv('../data/train_with_weather.csv', index=False)

In [8]:
merged_df

,index,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN
1,1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN
2,2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN
3,3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN
4,4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN
40987,40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN
40988,40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN
40989,40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN


# French Holidays

In [9]:
# Initialize SchoolHolidayDates
d = SchoolHolidayDates()

# Collect holiday dates for Zone C from 2020 to 2024
holiday_list = []
for year in range(2020, 2025):  # Loop through each year
    holidays_for_year = d.holidays_for_year_and_zone(year, 'C')
    for date in holidays_for_year.keys():
        holiday_list.append({'holiday_date': pd.to_datetime(date), 'is_holiday_zone_c': True})

# Create a DataFrame from the collected holiday dates
holiday_dates_zone_c = pd.DataFrame(holiday_list)

In [10]:
# Check that holiday_dates_zone_c has the 'is_holiday_zone_c' column
print(holiday_dates_zone_c.columns)  # Should include 'holiday_date' and 'is_holiday_zone_c'

# Merge holiday information with train data
merged_df_2 = pd.merge(merged_df, holiday_dates_zone_c, left_on='time', right_on='holiday_date', how='left')

# If 'is_holiday_zone_c' is missing after the merge, default to False
if 'is_holiday_zone_c' not in merged_df_2.columns:
    merged_df_2['is_holiday_zone_c'] = False
else:
    # Fill NaN in 'is_holiday_zone_c' with False
    merged_df_2['is_holiday_zone_c'] = merged_df_2['is_holiday_zone_c'].fillna(False)

# Drop unnecessary columns
merged_df_2 = merged_df_2.drop(columns=['air_quality_time', 'holiday_date','index'], errors='ignore')

# Display the merged DataFrame
merged_df_2

Index(['holiday_date', 'is_holiday_zone_c'], dtype='object')


,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_holiday_zone_c
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False


# Jours Feries

In [11]:
# Initialize French holidays for the range of years needed
france_holidays = holidays.France(years=range(2020, 2025))

# Assume merged_df_2 is already created and has an 'id' column with datetime values
# Convert 'id' to date-only format for easier holiday matching
merged_df_2['date_only'] = merged_df_2['time'].dt.floor('D')

# Add a new column in merged_df_2 to indicate if each date is a public holiday in France
merged_df_2['is_public_holiday'] = merged_df_2['date_only'].apply(lambda date: date in france_holidays)

# Create merged_df_3 by dropping the 'date_only' column after merging the holiday information
merged_df_3 = merged_df_2.drop(columns=['date_only'])

# Display the final merged DataFrame
merged_df_3

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_holiday_zone_c,is_public_holiday
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False


# Lockdown

In [12]:
lockdown_periods = [
    (pd.Timestamp(year=2020, month=3, day=12).date(), pd.Timestamp(year=2020, month=5, day=10).date()),
    (pd.Timestamp(year=2020, month=10, day=14).date(), pd.Timestamp(year=2020, month=12, day=10).date()),
    (pd.Timestamp(year=2021, month=3, day=31).date(), pd.Timestamp(year=2021, month=5, day=2).date())
]

# Add a 'lockdown' column with True if the date falls within any lockdown period, False otherwise
merged_df_3['lockdown'] = merged_df_3['time'].apply(
    lambda i: any(start <= i.date() <= end for start, end in lockdown_periods)
)

# Display the modified DataFrame
merged_df_3

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_holiday_zone_c,is_public_holiday,lockdown
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False


In [13]:
merged_df_3

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_holiday_zone_c,is_public_holiday,lockdown
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020-01-01,2.4,-0.8,4.6,0.0,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024-09-03,20.0,18.2,21.6,0.3,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False


# Weekend

In [14]:
# Add a new column indicating if the date is a weekend day ("Saturday" or "Sunday") or not
merged_df_3['is_weekend'] = merged_df_3['time'].apply(lambda x: x.weekday() >= 5)

# Display the modified DataFrame
merged_df_3

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,time,tavg,tmin,tmax,...,snow,wdir,wspd,wpgt,pres,tsun,is_holiday_zone_c,is_public_holiday,lockdown,is_weekend
0,2020-01-01 00:00:00,42.9,0.718,15.7,73.1,64.4,2020-01-01,2.4,-0.8,4.6,...,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False,False
1,2020-01-01 01:00:00,33.6,0.587,10.1,74.8,66.0,2020-01-01,2.4,-0.8,4.6,...,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False,False
2,2020-01-01 02:00:00,29.3,NaN,5.1,51.0,44.9,2020-01-01,2.4,-0.8,4.6,...,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False,False
3,2020-01-01 03:00:00,30.5,0.246,7.2,27.7,25.1,2020-01-01,2.4,-0.8,4.6,...,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False,False
4,2020-01-01 04:00:00,29.3,0.204,8.3,15.3,13.6,2020-01-01,2.4,-0.8,4.6,...,NaN,162.0,7.8,22.0,1031.4,NaN,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40986,2024-09-03 18:00:00,NaN,0.222,55.1,12.0,5.3,2024-09-03,20.0,18.2,21.6,...,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False,False
40987,2024-09-03 19:00:00,NaN,0.245,48.2,13.4,7.0,2024-09-03,20.0,18.2,21.6,...,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False,False
40988,2024-09-03 20:00:00,NaN,0.234,44.5,12.4,7.1,2024-09-03,20.0,18.2,21.6,...,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False,False
40989,2024-09-03 21:00:00,NaN,0.225,25.9,10.6,5.4,2024-09-03,20.0,18.2,21.6,...,NaN,239.0,8.2,28.0,1015.1,NaN,False,False,False,False


In [15]:
# Target Variable Extraction

train_processed = pd.read_csv('../data/train_with_weather.csv')

target_columns = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']

# Separate features and target for training
X_train = train_processed.drop(columns=target_columns)
y_train = train_processed[target_columns]

# Test set only has features, as targets are unknown
X_test = test_df.reset_index()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (40991, 13)
y_train shape: (40991, 5)
X_test shape: (504, 2)


Need to do the same preprocessing for X_test to get the same columns as X_train